In [ ]:
# Essential imports for image cartooning
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, filters, morphology, segmentation
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8')

print("🎨 Image Cartooning Laboratory - Part 1 Ready!")
print("✨ Transforming photographs into artistic cartoons")
print(f"OpenCV version: {cv2.__version__}")

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Advanced image reading and preprocessing functions
class ImageCartoonizer:
    """Comprehensive image cartooning system - Part 1"""
    
    def __init__(self):
        self.preprocessing_steps = []
        self.original_image = None
        self.processed_images = {}
    
    def load_image_safely(self, image_path_or_array):
        """Robust image loading with error handling"""
        try:
            if isinstance(image_path_or_array, str):
                # Load from file path
                image = cv2.imread(image_path_or_array)
                if image is None:
                    raise ValueError(f"Could not load image from {image_path_or_array}")
                # Convert BGR to RGB for matplotlib compatibility
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            else:
                # Assume it's already an array
                image = image_path_or_array.copy()
                
            # Ensure proper data type
            if image.dtype != np.uint8:
                image = (image * 255).astype(np.uint8) if image.max() <= 1.0 else image.astype(np.uint8)
                
            self.original_image = image
            print(f"✅ Image loaded successfully: {image.shape}")
            return image
            
        except Exception as e:
            print(f"❌ Error loading image: {e}")
            return None
    
    def analyze_image_properties(self, image):
        """Comprehensive image analysis for optimal preprocessing"""
        analysis = {
            'shape': image.shape,
            'dtype': image.dtype,
            'size_mb': image.nbytes / (1024 * 1024),
            'channels': len(image.shape),
            'color_channels': image.shape[2] if len(image.shape) == 3 else 1
        }
        
        if len(image.shape) == 3:
            # Color image analysis
            analysis['mean_rgb'] = np.mean(image, axis=(0, 1))
            analysis['std_rgb'] = np.std(image, axis=(0, 1))
            analysis['brightness'] = np.mean(image)
            analysis['contrast'] = np.std(image)
            
            # Color distribution
            hist_r = cv2.calcHist([image], [0], None, [256], [0, 256])
            hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
            hist_b = cv2.calcHist([image], [2], None, [256], [0, 256])
            analysis['histograms'] = {'r': hist_r.flatten(), 'g': hist_g.flatten(), 'b': hist_b.flatten()}
        else:
            # Grayscale analysis
            analysis['mean_gray'] = np.mean(image)
            analysis['std_gray'] = np.std(image)
            hist_gray = cv2.calcHist([image], [0], None, [256], [0, 256])
            analysis['histogram'] = hist_gray.flatten()
        
        return analysis
    
    def optimize_grayscale_conversion(self, image):
        """Optimized grayscale conversion with multiple methods"""
        methods = {}
        
        # Method 1: OpenCV standard conversion
        methods['opencv_standard'] = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Method 2: Weighted average (luminance)
        methods['luminance'] = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
        
        # Method 3: Simple average
        methods['average'] = np.mean(image, axis=2)
        
        # Method 4: Max channel (for high contrast)
        methods['max_channel'] = np.max(image, axis=2)
        
        # Method 5: Min channel (for shadows)
        methods['min_channel'] = np.min(image, axis=2)
        
        # Convert all to uint8
        for method_name, gray_img in methods.items():
            if gray_img.dtype != np.uint8:
                methods[method_name] = gray_img.astype(np.uint8)
        
        return methods
    
    def apply_median_blur_analysis(self, image, kernel_sizes=[3, 5, 7, 9, 11]):
        """Comprehensive median blur analysis with multiple kernel sizes"""
        blur_results = {}
        
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        for kernel_size in kernel_sizes:
            # Apply median blur
            blurred = cv2.medianBlur(gray, kernel_size)
            
            # Calculate noise reduction metrics
            noise_reduction = np.std(gray) - np.std(blurred)
            edge_preservation = self._calculate_edge_preservation(gray, blurred)
            
            blur_results[kernel_size] = {
                'image': blurred,
                'noise_reduction': noise_reduction,
                'edge_preservation': edge_preservation,
                'mean_intensity': np.mean(blurred),
                'std_intensity': np.std(blurred)
            }
        
        return blur_results
    
    def _calculate_edge_preservation(self, original, processed):
        """Calculate how well edges are preserved after processing"""
        # Calculate edge strength in both images
        edges_orig = cv2.Canny(original, 50, 150)
        edges_proc = cv2.Canny(processed, 50, 150)
        
        # Calculate edge preservation score
        edge_pixels_orig = np.sum(edges_orig > 0)
        edge_pixels_proc = np.sum(edges_proc > 0)
        
        if edge_pixels_orig == 0:
            return 0
        
        preservation_score = edge_pixels_proc / edge_pixels_orig
        return preservation_score

# Load test images for cartooning
def load_test_images():
    """Load diverse test images for cartooning experiments"""
    images = {
        'Astronaut': data.astronaut(),
        'Coffee': data.coffee(), 
        'Coins': data.coins(),
        'Camera': data.camera(),
        'Chelsea': data.chelsea()
    }
    return images

# Initialize cartooning system
cartoonizer = ImageCartoonizer()

# Load test images
test_images = load_test_images()

print("🔍 Analyzing test images for cartooning...")

# Analyze each test image
for name, img in test_images.items():
    print(f"\\n📊 Analyzing {name}:")
    
    # Load image safely
    loaded_img = cartoonizer.load_image_safely(img)
    
    if loaded_img is not None:
        # Analyze image properties
        analysis = cartoonizer.analyze_image_properties(loaded_img)
        
        print(f"  Shape: {analysis['shape']}")
        print(f"  Size: {analysis['size_mb']:.2f} MB")
        print(f"  Brightness: {analysis['brightness']:.1f}")
        print(f"  Contrast: {analysis['contrast']:.1f}")
        
        # Test grayscale conversion methods
        gray_methods = cartoonizer.optimize_grayscale_conversion(loaded_img)
        
        # Test median blur
        blur_results = cartoonizer.apply_median_blur_analysis(loaded_img)
        optimal_kernel = max(blur_results.keys(), 
                           key=lambda k: blur_results[k]['edge_preservation'])
        print(f"  Optimal median blur kernel: {optimal_kernel}")

print("\\n✅ Image analysis complete!")
